In [25]:
from llama_cpp import Llama

llm = Llama.from_pretrained(
    repo_id="hugging-quants/Llama-3.2-3B-Instruct-Q8_0-GGUF",
    filename="*q8_0.gguf",
)

llama_model_loader: loaded meta data with 30 key-value pairs and 255 tensors from /home/petko/.cache/huggingface/hub/models--hugging-quants--Llama-3.2-3B-Instruct-Q8_0-GGUF/snapshots/7ef7efff7d2c14e5d6161a0c7006e1f2fea6ec79/./llama-3.2-3b-instruct-q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llama 3.2 3B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Llama-3.2
llama_model_loader: - kv   5:                         general.size_label str              = 3B
llama_mod

{'id': 'chatcmpl-9b2d6f3f-a330-4e94-b6df-0c07cb6315bf', 'object': 'chat.completion', 'created': 1732447850, 'model': '/home/petko/.cache/huggingface/hub/models--hugging-quants--Llama-3.2-3B-Instruct-Q8_0-GGUF/snapshots/7ef7efff7d2c14e5d6161a0c7006e1f2fea6ec79/./llama-3.2-3b-instruct-q8_0.gguf', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'Борисов е български политик, дългогодишен член на Българска социалдемократическа партия (СДП).'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 19, 'completion_tokens': 42, 'total_tokens': 61}}


In [ ]:
output = llm.create_chat_completion(
      messages = [
          {
              "role": "user",
              "content": "Кой е Бойко Борисов?"
          }
      ]
)

print(output)

In [26]:
output['choices'][0]['message']['content']

'Борисов е български политик, дългогодишен член на Българска социалдемократическа партия (СДП).'

In [64]:
def generate_prompt(input_text):
    prompt = f"""
    Генерирай въпрос-отговор двойки на български език със следната JSON структура:
        {{
            "question": "<Въпрос на български>",
            "answer": "<Точен отговор на български>",
            "context": "<Контекст от текста на български>"
        }},
        
        Генерирай въпроси само на български, които могат да бъдат отговорени директно чрез контекста. Избягвай неясни или спекулативни въпроси. Увери се, че отговорите са кратки и основани на предоставения текст.

        Това е текста:
        {input_text}
        """
    return prompt


In [65]:
def has_valid_response_structure(response):
    return "question" in response and "answer" in response and "context" in response

In [68]:
import os
import ollama
import json
from tqdm import tqdm

MODEL = 'todorov/bggpt' # model to extract question-answer pairs
DATA_PATH = './archive'

output_file = 'question_answer_pairs.json'

if not os.path.exists(output_file):
    with open(output_file, 'w', encoding='utf-8') as file:
        json.dump([], file)

dirs_to_process = []
for root, dirs, files in os.walk('./archive'):
    dirs_to_process.extend(dirs)

with tqdm(total=len(dirs_to_process), desc="Processing directories") as pbar:
    for root, dirs, files in os.walk(DATA_PATH):
        for file in files:
            if file.endswith('.txt'):
                with open(os.path.join(root, file), 'r') as f:
                    file_content = f.read()
                
                prompt = generate_prompt(file_content)
                
                response = ollama.chat(
                    model=MODEL,
                    format="json",
                    messages=[{'role': 'user', 'content': prompt}],
                )
                
                try:
                    with open(output_file, 'r', encoding='utf-8') as file:
                        data = json.load(file)
                except FileNotFoundError:
                    data = []

                if has_valid_response_structure(response.message.content):
                    data.append(response.message.content)

                    with open(output_file, 'w', encoding='utf-8') as file:
                        json.dump(data, file, ensure_ascii=False, indent=4)
        
        pbar.update(1)

Processing directories: 9911it [15:03:24,  5.47s/it]                               
